In [ ]:
import time
import random
import matplotlib.pyplot as plt
import numpy as np

# AES and DES Functions (as in your original code)
inverse_des_sbox = {v: k for k, v in enumerate(des_sbox)}
inv_aes_sbox = {v: k for k, v in enumerate(aes_sbox)}

def aes_substitute(byte):
    return aes_sbox[byte]

def inverse_aes_substitute(byte):
    return inv_aes_sbox[byte]

def aes_shift_rows(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    return (byte2 << 8) | byte1

def inverse_aes_shift_rows(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    return (byte1 << 8) | byte2

def gf_mult(a, b):
    p = 0
    for _ in range(8):
        if b & 1:
            p ^= a
        hi_bit_set = a & 0x80
        a <<= 1
        if hi_bit_set:
            a ^= 0x1B
        b >>= 1
    return p & 0xFF

def aes_mix_columns(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    mixed1 = gf_mult(byte1, 2) ^ gf_mult(byte2, 3)
    mixed2 = gf_mult(byte2, 2) ^ gf_mult(byte1, 3)
    return (mixed1 << 8) | mixed2

def inverse_aes_mix_columns(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    inv_mixed1 = (gf_mult(byte1, 0x0E) ^ gf_mult(byte2, 0x0B) ^
                  gf_mult(byte1, 0x0D) ^ gf_mult(byte2, 0x09))
    inv_mixed2 = (gf_mult(byte2, 0x0E) ^ gf_mult(byte1, 0x0B) ^
                  gf_mult(byte2, 0x0D) ^ gf_mult(byte1, 0x09))
    return ((inv_mixed1 & 0xFF) << 8) | (inv_mixed2 & 0xFF)

def des_sbox_substitution(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    return (des_sbox[byte1] << 8) | des_sbox[byte2]

def inverse_des_sbox_substitution(value):
    byte1 = (value >> 8) & 0xFF
    byte2 = value & 0xFF
    return (inverse_des_sbox[byte1] << 8) | inverse_des_sbox[byte2]

def des_permutation(value):
    permuted = 0
    permutation_table = [15, 6, 11, 3, 8, 0, 12, 4, 7, 14, 1, 10, 2, 9, 13, 5]
    for i, bit_pos in enumerate(permutation_table):
        if value & (1 << bit_pos):
            permuted |= (1 << i)
    return permuted

def inverse_des_permutation(value):
    inverse_permuted = 0
    permutation_table = [15, 6, 11, 3, 8, 0, 12, 4, 7, 14, 1, 10, 2, 9, 13, 5]
    for i, bit_pos in enumerate(permutation_table):
        if value & (1 << i):
            inverse_permuted |= (1 << bit_pos)
    return inverse_permuted

def des_key_mixing(value, round_key):
    return value ^ round_key

def generate_round_keys(base_key, num_rounds=12):
    return [(base_key ^ (i * 0x9F37)) & 0xFFFF for i in range(num_rounds)]

def aes_des_encrypt(data, round_keys, num_rounds=12):
    left, right = (data >> 16) & 0xFFFF, data & 0xFFFF
    for round_num in range(num_rounds):
        round_key = round_keys[round_num]

        left = aes_substitute(left & 0xFF) | (aes_substitute((left >> 8) & 0xFF) << 8)
        left = aes_shift_rows(left)
        left = aes_mix_columns(left)
        left ^= round_key

        right = des_sbox_substitution(right)
        right = des_permutation(right)
        right = des_key_mixing(right, round_key)

        left, right = right, left
    return (left << 16) | right

def aes_des_decrypt(data, round_keys, num_rounds=12):
    left, right = (data >> 16) & 0xFFFF, data & 0xFFFF
    for round_num in reversed(range(num_rounds)):
        round_key = round_keys[round_num]

        left, right = right, left
        right = des_key_mixing(right, round_key)
        right = inverse_des_permutation(right)
        right = inverse_des_sbox_substitution(right)

        left ^= round_key
        left = inverse_aes_mix_columns(left)
        left = inverse_aes_shift_rows(left)
        left = inverse_aes_substitute(left & 0xFF) | (inverse_aes_substitute((left >> 8) & 0xFF) << 8)
    return (left << 16) | right

# RSA Key Generation
def generate_rsa_keys():
    p, q = 104729, 99991
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    d = mod_inverse(e, phi)
    return (n, e, d)

def mod_inverse(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

def rsa_encrypt(data, e, n):
    return pow(data, e, n)

def rsa_decrypt(data, d, n):
    return pow(data, d, n)

# Performance Measurement
data_sizes = [500, 1000, 1500, 2000, 2500]
aes_des_enc_times, rsa_enc_times = [], []
aes_des_dec_times, rsa_dec_times = [], []
total_enc_times = []  # Stores total encryption time (AES-DES + RSA)

n, e, d = generate_rsa_keys()
base_key = 0xA3F4B67C
round_keys = generate_round_keys(base_key)

for size in data_sizes:
    data = random.randint(0, 0xFFFFFFFF)

    # AES-DES Encryption
    start = time.perf_counter()
    aes_des_encrypted = aes_des_encrypt(data, round_keys)
    aes_des_time = time.perf_counter() - start
    aes_des_enc_times.append(aes_des_time)

    # RSA Encryption
    start = time.perf_counter()
    rsa_encrypted = rsa_encrypt(aes_des_encrypted, e, n)
    rsa_time = time.perf_counter() - start
    rsa_enc_times.append(rsa_time)

    # Store total encryption time (AES-DES + RSA)
    total_enc_times.append(aes_des_time + rsa_time)

    # RSA Decryption
    start = time.perf_counter()
    rsa_decrypted = rsa_decrypt(rsa_encrypted, d, n)
    rsa_dec_times.append(time.perf_counter() - start)

    # AES-DES Decryption
    start = time.perf_counter()
    aes_des_decrypted = aes_des_decrypt(rsa_decrypted, round_keys)
    aes_des_dec_times.append(time.perf_counter() - start)

# Calculate Averages
avg_aes_des_enc = sum(aes_des_enc_times) / len(aes_des_enc_times)
avg_rsa_enc = sum(rsa_enc_times) / len(rsa_enc_times)
avg_aes_des_dec = sum(aes_des_dec_times) / len(aes_des_dec_times)
avg_rsa_dec = sum(rsa_dec_times) / len(rsa_dec_times)

# --- PLOT FIGURE 1: AVG ENCRYPTION TIME (AES-DES vs. RSA) ---
plt.figure(figsize=(6, 4))
plt.bar(["AES-DES", "RSA"], [avg_aes_des_enc, avg_rsa_enc], color=["blue", "orange"])
plt.xlabel("Encryption Operations")
plt.ylabel("Time (sec)")
plt.title("Avgerge Encryption Operations")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.show()

# --- PLOT FIGURE 2: AVG DECRYPTION TIME (AES-DES vs. RSA) ---
plt.figure(figsize=(6, 4))
plt.bar(["AES-DES", "RSA"], [avg_aes_des_dec, avg_rsa_dec], color=["blue", "orange"])
plt.xlabel("Decryption Operations")
plt.ylabel("Time (sec)")
plt.title("Avgerage Decryption Operations")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.show()

# --- PLOT FIGURE 3: DATA SIZE vs TOTAL ENCRYPTION TIME ---
plt.figure(figsize=(8, 5))
plt.bar([str(s) + " bytes" for s in data_sizes], total_enc_times, color="#876FD4")
plt.xlabel("Data Size")
plt.ylabel("Total Encryption Time (sec)")
plt.title("Total Encryption Time vs. Data Size")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.show()